<a href="https://colab.research.google.com/github/zihadbappy/Autumn20labs/blob/master/Final_LSTM_BiLSTM_CNN%2BLSTM_CNN%2BBiLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [486]:
!git clone https://github.com/zihadbappy/bnfakedata.git

fatal: destination path 'bnfakedata' already exists and is not an empty directory.


In [487]:
import pandas as pd
Dataset= pd.read_csv('/content/bnfakedata/bnfakefinal.csv')

In [488]:
df = pd.DataFrame(Dataset)

In [489]:
df = pd.DataFrame(Dataset)


In [490]:
df.groupby('label').agg('count')

,headline,content
label,,
0,2790,2790
1,2790,2789


In [491]:
df=df.dropna()

In [492]:
X=df.content
Y=df.label

In [493]:
Y.value_counts()

0    2790
1    2789
Name: label, dtype: int64

In [494]:
X.shape

(5579,)

In [495]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

Onehot Representation


In [496]:
voc_size=96000

In [497]:
onehot_repr=[one_hot(words,voc_size)for words in X]

In [498]:
len(onehot_repr)

5579

In [499]:
news_length=300
embedded_news=pad_sequences(onehot_repr,padding='pre',maxlen=news_length)
print(embedded_news)

[[    0     0     0 ... 53594 65523 16909]
 [    0     0     0 ...  3635 10660  1667]
 [62387 92709 21482 ... 77144 10660  1667]
 ...
 [    0     0     0 ... 26122 53221  7967]
 [    0     0     0 ... 53336  7000 69174]
 [    0     0     0 ... 48232 93491 84295]]


In [500]:
embedded_news[0]


array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0, 86335, 75502, 24216, 64487, 16297, 58603,
       78941, 82259, 93931, 71005, 62045, 65685, 58191, 23735, 40106,
       83277,  5242, 10883, 81471, 75256, 44324, 16175, 54032, 79483,
       30398, 40106, 63670, 61299, 13375,  5242, 10883, 26113, 12127,
       53047, 11210, 67710, 65194, 79166, 83002, 45768, 67749, 91198,
       81344, 81904, 59753, 81471, 17375, 89391, 94145,  6337, 81471,
       75256, 44324, 74692, 66779, 10883, 83277, 12127, 86460, 72372,
       83553, 83553,

In [501]:
len(embedded_news),Y.shape

(5579, (5579,))

In [502]:
import numpy as np
X_final=np.array(embedded_news)
y_final=np.array(Y)

In [503]:
X_final.shape,y_final.shape

((5579, 300), (5579,))

In [504]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.25, random_state=99)

# **LSTM MODEL**

In [505]:
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=news_length))
model1.add(LSTM(100))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=8,batch_size=64)

Model: "sequential_58"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_58 (Embedding)    (None, 300, 40)           3840000   
                                                                 
 lstm_58 (LSTM)              (None, 100)               56400     
                                                                 
 dense_58 (Dense)            (None, 1)                 101       
                                                                 
Total params: 3,896,501
Trainable params: 3,896,501
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/8
66/66 [==============================] - 4s 29ms/step - loss: 0.4213 - accuracy: 0.8260 - val_loss: 0.1787 - val_accuracy: 0.9427
Epoch 2/8
66/66 [==============================] - 1s 22ms/step - loss: 0.1116 - accuracy: 0.9694 - val_loss: 0.1417 - val_accuracy: 0.9520
Epoch 3/8
66/66 [=====

# **CNN+LSTM MODEL**

In [506]:
from keras.layers import Dense,Conv1D,MaxPooling1D

In [507]:
embedding_vecor_length = 40
model2 = Sequential()
model2.add(Embedding(voc_size, embedding_vecor_length, input_length=news_length))
model2.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model2.add(MaxPooling1D(pool_size=2))
model2.add(LSTM(100))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model2.summary())
model2.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=8,batch_size=64)

Model: "sequential_59"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_59 (Embedding)    (None, 300, 40)           3840000   
                                                                 
 conv1d_29 (Conv1D)          (None, 300, 32)           3872      
                                                                 
 max_pooling1d_29 (MaxPoolin  (None, 150, 32)          0         
 g1D)                                                            
                                                                 
 lstm_59 (LSTM)              (None, 100)               53200     
                                                                 
 dense_59 (Dense)            (None, 1)                 101       
                                                                 
Total params: 3,897,173
Trainable params: 3,897,173
Non-trainable params: 0
___________________________________________

LSTM and CNN+LSTM evaluation

In [508]:
y_pred1=model1.predict(X_test)
y_pred2=model2.predict(X_test)

In [509]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [510]:
print("LSTM accuracy:%.4f"%accuracy_score(y_test,np.around(y_pred1)))
print("CNN+LSTM accuracy:%.4f"%accuracy_score(y_test,np.around(y_pred2)))

LSTM accuracy:0.9491
CNN+LSTM accuracy:0.9577


In [511]:
print("LSTM:\n", classification_report(y_test,np.around(y_pred1),digits=4))
print("\nCNN+LSTM:\n", classification_report(y_test,np.around(y_pred2),digits=4))

LSTM:
               precision    recall  f1-score   support

           0     0.9524    0.9456    0.9490       698
           1     0.9459    0.9527    0.9492       697

    accuracy                         0.9491      1395
   macro avg     0.9491    0.9491    0.9491      1395
weighted avg     0.9491    0.9491    0.9491      1395


CNN+LSTM:
               precision    recall  f1-score   support

           0     0.9469    0.9699    0.9582       698
           1     0.9691    0.9455    0.9572       697

    accuracy                         0.9577      1395
   macro avg     0.9580    0.9577    0.9577      1395
weighted avg     0.9580    0.9577    0.9577      1395



In [512]:
print("LSTM confusion matrix:\n ",confusion_matrix(y_test,np.around(y_pred1)))
print("\nCNN+LSTM confusion matrix:\n",confusion_matrix(y_test,np.around(y_pred2)))

LSTM confusion matrix:
  [[660  38]
 [ 33 664]]

CNN+LSTM confusion matrix:
 [[677  21]
 [ 38 659]]


#**Bi-LSTM MODEL**

In [513]:
embedding_vector_features=40
model3=Sequential()
model3.add(Embedding(voc_size,embedding_vector_features,input_length=news_length))
model3.add(Bidirectional(LSTM(100)))
model3.add(Dropout(0.3))
model3.add(Dense(1,activation='sigmoid'))
model3.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model3.summary())
model3.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=8,batch_size=64)

Model: "sequential_60"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_60 (Embedding)    (None, 300, 40)           3840000   
                                                                 
 bidirectional_29 (Bidirecti  (None, 200)              112800    
 onal)                                                           
                                                                 
 dropout_15 (Dropout)        (None, 200)               0         
                                                                 
 dense_60 (Dense)            (None, 1)                 201       
                                                                 
Total params: 3,953,001
Trainable params: 3,953,001
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/8
66/66 [==============================] - 6s 49ms/step - loss: 0.5434 - accuracy: 0.7500 - val_

## **CNN+BiLSTM MODEL**

In [514]:
embedding_vecor_length = 40
model4 = Sequential()
model4.add(Embedding(voc_size, embedding_vecor_length, input_length=news_length))
model4.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model4.add(MaxPooling1D(pool_size=2))
model4.add(Bidirectional(LSTM(100)))
model4.add(Dense(1, activation='sigmoid'))
model4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model4.summary())
model4.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=8,batch_size=64)

Model: "sequential_61"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_61 (Embedding)    (None, 300, 40)           3840000   
                                                                 
 conv1d_30 (Conv1D)          (None, 300, 32)           3872      
                                                                 
 max_pooling1d_30 (MaxPoolin  (None, 150, 32)          0         
 g1D)                                                            
                                                                 
 bidirectional_30 (Bidirecti  (None, 200)              106400    
 onal)                                                           
                                                                 
 dense_61 (Dense)            (None, 1)                 201       
                                                                 
Total params: 3,950,473
Trainable params: 3,950,473
N

In [515]:
y_pred3=model3.predict(X_test)
y_pred4=model4.predict(X_test)

In [516]:
print("BiLSTM accuracy:%.4f"%accuracy_score(y_test,np.around(y_pred3)))
print("CNN+BiLSTM accuracy:%.4f"%accuracy_score(y_test,np.around(y_pred4)))

BiLSTM accuracy:0.9527
CNN+BiLSTM accuracy:0.9620


In [517]:
print("BiLSTM:\n",classification_report(y_test,np.around(y_pred3),digits=4))
print("\nCNN+BiLSTM:\n",classification_report(y_test,np.around(y_pred4),digits=4))

BiLSTM:
               precision    recall  f1-score   support

           0     0.9305    0.9785    0.9539       698
           1     0.9773    0.9268    0.9514       697

    accuracy                         0.9527      1395
   macro avg     0.9539    0.9527    0.9527      1395
weighted avg     0.9539    0.9527    0.9527      1395


CNN+BiLSTM:
               precision    recall  f1-score   support

           0     0.9722    0.9513    0.9616       698
           1     0.9522    0.9727    0.9624       697

    accuracy                         0.9620      1395
   macro avg     0.9622    0.9620    0.9620      1395
weighted avg     0.9622    0.9620    0.9620      1395



In [518]:
print("BiLSTM confusion matrix:\n",confusion_matrix(y_test,np.around(y_pred3)))
print("\nCNN+BiLSTM confusion matrix:\n",confusion_matrix(y_test,np.around(y_pred4)))

BiLSTM confusion matrix:
 [[683  15]
 [ 51 646]]

CNN+BiLSTM confusion matrix:
 [[664  34]
 [ 19 678]]
